In [1]:
text = ['GLTNAFIASAPAREVRYDGVITPANANYRFMGGDKGGSLTVGSHLTGSNMVTIGPMGVVVFTNNNDYTGNTFIMGGGTLQLGSNTAWGSLPNSGIGIGGAVSNYGSLLIARSDTYTLTNYIVGSGTLVKNGTNVLIVTANNIYTGGTYINGGTVVAGATNAFGTGPMIVRSYAGQFNPGFSSVLDVNGQQLVIGSLLGQVGARVTDNSANGGITMLTINQGTTYTYSGTIGDGENTALALTKIGGGILTLASSNDYTGGTFVSAGILRATLGVGLPTTGNLTLATNGVFETGMNITNALGNGDGQIQILAGGAAGFSPQNVYGLESFAPTGIVVNLGGQGETVVWGSQFFNPNQFVLNAGSAGTNNILTFVNPLDLNGATRSVSVVGSTGILAGTISSSSGLANFTKEGAGWLNVVAPMNFTGAVTVNGGTLMLTASNTYTGGVTINAGTLYAYSDANFGVAGAAITLSNNAAVLRTTNQFTLNRALVNAANGAINVDQYATLTVTGQVSGGGGLFTKAGFGTLILAPDRSTGGNNYFGSNILSAGTILVDFSASTPLGHAGLDGSAGGTAAIGPTGELDGNFLGWVSGKALQTASGTNIPAKQFLGAIVLATNSSKDLNFNNYPMLTNAFLGASEGTWSYYGVLNNWTQSTLGNAFFGTAVTNIYQLGGGNGTLRYEPQITGTNNLIIGPVGGNGTVWLPNQNTFTGTVLVAGGTLRAQEGVGLPSTAALMLSGGVFEVMSQMGGTNFVRGLGTGVGQVWLATNAPGGFSTINGPTTIKFGGASTNIVWGSVGSAFSPTVLILNTNTADSKLTLDNGIDLNGVTRTVMVGGNTGEISGPIYTSTGNPGLIKSGTGTLLLSGNNFHWGNTINAGNVVLGSALGLGNIGVGADASALLQLNYGTALDLNGYSIVIQGLGMTANSILTDNSATPGVSVVNLYQAQGVNWNNQLAGLITDGVNGRQVGLTIGMAQRNNDGRYWIPYTNAYSGGTLIYGGSVFVGHTNSLGTGSIRLVGGGGLGPMYKPLDQDFLDYVNERAGATGVVSGALLMQGGNPRFEDLGRFVTNNFDFSTGAISNAFLGVGYGGTVWYSGTITPAADAWRLGGGGGILIPSQIYSNVGANVMNMEIGAKAGNTADRGGQYWFGSSAVYFTNSMTYSGTTLLRQGAALVLSNAYGTAITGSTNIILGDSLTAVGNVAVMILAANEQINSNAVVHFDAISGRQGYLTLNGFNQTLKGIDDPTGYGVIENREPMHVTGNNISMLTLSNDVDNVFYGYFRDYASGVAPMGNPVLGLTKNGTGTLTLGGAGITYSGPTIINQGTLVLSDWGTMSDINRGLGVGIISNNSALILDVNWAGRISGLSYPVVGTGVVMKTGAQTLKLEFDGAGAPTSNILDQTAPFTMAGGVLAVIGGPAGGVNPAFTTQNFDNFILAPGYSIVMATNGPLGAPVALNLAATLPLTRSEGAVLMVVETAGQSVVRGAPANGPGGIIGGWALSGYNDWAVNNGAGIFTNYSAYAFVTNYASALGSWAPTSNVTLAANAFIAGNGGAVNTVRFTNNNMNVSAILNGTNVIVSGGILFGTNVAISQFGANNAAAGLYGGAITSGTNELHILQNTFYIPNQDRRNVYLDTIGSKIVDNGATSVTVVLSSPPIAVNLYGPAPAGIRLANSENSFSGGLVLNGVQVQVDEEGALGAVPSTPVTNIIANAGYNWLRFPNATTLSANRGIYIGTGAVLLLEARYAADNAVGATNLFVPGTISGPGRLIAGSLQEDSRIVLTGTNTLTGVVEVWGVLAAQDGVGLPSTANLILNSDRQARNVQNYGAGVLEMTGTFSRALGSGPGQFQIWTDNRPSQQWAVNQGGFAAVGAPLTVNVGGDGRPLVYGQTFFNPGSLQFGDLYTTHPVTWVNPLDLAGAQRTFTVHVGTGIISGELSSAKTISINGSPQYSGGGVAKYGGGVLVLPVTNTYRGLTTIGLGTLRAQDGVGLPSLSPLIMVNDNGWGTMGFPNSAGTASAPSMSALEMTGVFTRPLLNTLGAVNPGFGYGITEGGVSLLGVGSVGFSAKGGDLTVNIGGNGETLVWSNTAANALELATAQNNATVTIVRGGNTDDLRVGMNVQGFAIRNNTYISAITGDYTFTMSQNSVDAASSISTNWSTFSTFNPGFLLLNGPEADSTVTFVNNLDFNGLNRGISVAGSPVGMIGGGNTGILSGAIVTSTNFWAGMPVALIKAGYGTLVLQATNKLDNGVQLNAGELEITSHANIGGQNSPIYFNGGILRLANPAMTTLDGTHPINATNFNGGFDIVDGATFTVANVLAGNNAGNALFKVGSGTLKLSGLAVNGPLVVNGGTLDLNGVNVAAGGLTGATSTIITDNGSVAGVSAITLGSTANITTNGVNVFAGTITNGANRQIALVINSGGLVLSGQTNYYSGGTYVNAGNLLIASTDSIGTGPLVLTNASSGIGLYNADLTQSFLNWVGTRAVGGFQGTLLLTSNSSASFDFNNANYRGAFLGVAYGFTSVYSGTITPAGGVYRLGGGGGTLIPEQVFSGGNSMEVGPAGGIMAPGFYPPTGGAPTTNSVVIFTNTYSYTGSTLLRQGTALILSNTTGSVIPGNLQMGDAFGLPAPASYAVLQLGANEQIADTGVVSFGGIANRWAYFKLMGHDETIGGLSDQAGVGVIENQEGEWVNRAGMLTVNNNADFSFNGYMRDRSGGYGAGYLMLTKGGTGMQSLSGANITYSGQTIVNDGTLRLADVTGFASRYITNAGTVVFDGASAVTLNNTPFLVGTGAVVKAGTSTLTVNYAYDYSGPTVINGGTLQLAGTGTILNTSTITINAGGALTLNNAVATGINTNRIADNTPVIMNGGTINLLIDTVTNANAQMRGVTIESGNNTINVGQAGANLTNRLVIGNILTNRGAGATVNFTGTGLGQGNNRAVIQFTAAPTLTNGILAPWAIVNGAGSGANWATYNSTLNSISNYTGYYYAGADGAATWVSTTNIYTTAGLTLGSSRTINSLVVSNGTLNLGGNTLTIRSGGLMSVGGVTSLISGGNLTAGEAGVGGELIYFQYNIGAAPTNQVTANIVNNGAGPVTFVKSGPAALALTYGTNTYSGGTIVDQGMLMVGYNGNMATSGSTVIGLGSGTLTVQPGGSVIFAPGSGSTYYFTNAVVLNGGTIGNNGGGNGGIFHFTGPMTLNQPLGFAGNASNSTIAVGGFASQDLYFDGTISGNGGLRIVSAGNVGAAFLTTNNTYSGGTLVDGSSFLFLGSTTNLSTSATVTNTLGSGLLSAPGIAGGVGAAYAMDQGFLDFINARASSPTSVVNLLLFSGTVANDLDFSTEANLVNTYLGMRPWAGNVTYNGSITPATNTYRFNAFAGNGSLVIGSLLTNAASGNNNMLIGPGGTVVLTNDNTLSGWTYVTNGGTLQIGTNSTYGDLGNGVVTNMGTVLFARSDTYTNYNLFVGGSGTILKQTNNSVLQLAGNNGVYNGAITVNGGRLQAGATNAFGNGAGVMTLNNARTVLDLNGYNAIIGTPAVNANTVITDDGTNGRVTTLSITNAGTSTITGVIADGQTNTIALAKYGAGLLNLLNTNTFSGDVLVAGGVLRATRGFSIGTNLNISLNGGVFENSFRVANNLGTADNEIQLPYGISGFSAFATNVIVDLGGTNAVLTWGSAGFNPVNFVLNESTATATLVLSNKVDLAGLSRAFSVNANTAMVGGVIFNSGSEPAGIIKYGAGTLQMMGSNLYNGLTVLNGGTLYINSDSNLGAAGYGISFNASATLRVTNDITLNNRPITIGSGMAPTFNIDPFSALTLSNPIAAMGVAGNVTKTGLGRLILLGTNAYTSQTIIQQGTVLLGSTNVLYGGAGLINLNAALGAIGTTGAVDQSFVEWVGTHSVNPVNGAIVLGNHSTNALNFNFTNAYGAYRNYNWLSNAFLGAADGTWTYSGELKTWTNSVLSVNLGLGTNTLLLGGGSGTLDFAPTIGGNTNVIIGPVGGYGVVQFTNYNAFTGSLAVIGGTFRAATNIGMPNTANLMLTGGVFEVLGGGNLTSALGIGAGKIQLAAGAASGFSTTNGNLTINFGGAGALVQWNTNNFFAPSALVLGTSTTSGVITFVNPIDVTNAARVINVAGGTAVLAGYLTNSAGTPDLTKWGTGTLIFSNASGSTLTAVPRLRQGTTIVETGAVLRLATGISIGQTGGDVATLTVRGNGQLIAVASDFNMMDTGNGLAVVNVQDNALVQIQRLRMARAANAVAVFNQTGGLVQSRSGGGDWAIGETAGGFGYYNLSGGLLVNSNNPIRIGSGTWGQMDMTGGIVTNNQNTTLSFAGGYMGVLNVMGGTFNQTANRLIVGDAGSGYGVLNVGGSGVVNATGGGIRLGWNANAAGVVNLSDGGTIIASVVEQGAGSGDVFNFDGGTLKAAAAGSTFTFTNGGQAYVYDGGAVIDTTNLSMTIAQALLAADGYGIINIPVAYGGAGYMAPPIVKFTGGGGSNATAYAQIDPVSGSVTNIVVTSPGTGYTNAPIIQLVMPGFAGQGGPTYAAVAGTALLGTNVDGGLTKLGAGTLTLTATNTYVGPTVISEGILQISAGGLLGSGMIADNAALFFNSAFTNTFGNFINGTGYIAQIGTGVTVLTNVNTYGGSTYVSNGVLRADNGVGLSTASNLVLNGGVIEMGAAFTRDIGATNNAVSLAGAAGFSAYGAPVTVTLGGGTTLVWGSTFFAPTNLILNAASANDTLTFANGLDLAGFNRTISVHAATGVISGVVSGPALSKLGTGTLSLSGANTFTNMHILGGTLQIGANNTLWTNSIVTFGDNTNSPNMAGALDLSLFSQQIGGLIVATTNYYATNSIAIGSGQQLTVLGNVKLGVDAFSGAPTNGGFSKLVMSGAGSFIVTGTSFRVSAGDNTSNGIQALLDMSGLATANINLGATGVFTVGDMQTLQANVNDSSTVLLASNSTITANTLYLGNGNRIGQQVMRLGSGTNILNANTINVGSPNGGRSFGSMVFNGSTGILTVRAADGVGRAAFNVGAGSVGTAAGGTNTVDFTGHTVDLFLSTLNVGEENRQGLSTNIFSFDMGTLDATGMRIGNRPAGQFRTENGFAGIVNLGGGVVNIGDKGIVMGTSYGTNYGTNMAVLNITGDAAVTVASNIVMLTQNSRSSVVYSTINVLGSSSLTMLSNIITGVDGAGATTRVATININGGTVDMSGHQIGVSGGLVDNLNLISGTLQNLGEFNAGGTIFKTNAGLFTLAGSNSYSGATIVTNGTLILAGSVVNSVSLGVTNATVNFDGTNHAFGTMAFGNGAIINFARGDVSSGEMDFRGSAIWTVTNGFAGNTSLGSLLVNNSASPVWNLSGSTVNMANLFGYAAGGGGSLWITNGALNVSSTVSNRMTMTISNAVVNANRFVNGDTTDRALTNYGVLTVAGAIVNQGGSTLENAGAITAGVLQVTSGIITNTGSIAAGAVTNSGSIFVTGGSLTVSGSITNTASGNLNVVNGIVRAGNLYGGIFLDPSTGYFGNVVLDNDHYTIVDAGSAMHLSGNMDNQLTNRVAEGSTLAPTWYGFQGTFLFTNAPSAGAEGTQQVEVASIRTEEASMATTNFFIGSFVVGDPATGNKTYVQLVDNRVNAPADAVLPSSEILAASNLLIWTSSTVDWNNRSGFAYNLSNAGTMLWTNAGNPDGTVVRLDVVNTFTNRGLMEIGNGTVLQMSNAFLNGSTANIALFNGGVLTNFATGGVLTNSGTIYGDGLVAPAISNGVGGVVAAVTGHLLLSGGIADNVNYGMLAATNSGSTLSVNNALLTNASGATIGLAGGTFTMVGAGSNVVNHGSVAGFGTVAAVIGTEADGTVVATGGVLNLTAGFTNGVAGTPVNAGLLAALGVNSELVLNQAITNVGSIQLNNATAVFTAPSVNNAGAISGHGTFNSALDNAGTVTSGNSGQTLTFSQSVNNQAVGTMTAQNGGSFWFSNAVANAGSITVRDAGTATFGGAVTNNGTINVQGQSTLRFTAGLTNNGTLAFGPTVSPSTAIITGTLLIGQNGVITMGSLTNNTLVVQGDFLNRSTNNQASDTRNGTVVVGGTTPLTGLGAATNTFEVAGNDMGAVYAGFNTNFAIGTLNITNHTSFVGNGLSGDPAQYVDVLHLFSGATMKLSALTIYVGTQFIYEDSNGMKVFNGGAGDAITAANKDSLGLANVFLDAGGQIVFVPEPSTGVLIGLGLAALAGSRRRKKAARC\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFPLSPAQLGIWYAQHLDPQVPITIAQYVDLHGALDVEVLERASIDASHELGSGFLRIVERDGEPLQYV\n']

In [3]:
from datasets import Dataset
train_dataset = Dataset.from_dict({'sequences': text})
train_dataset

Dataset({
    features: ['sequences'],
    num_rows: 2
})

In [4]:
from transformers import PreTrainedTokenizer
import sentencepiece as spm
tokenizer_path = '/data/rozen/home/e0833634/lama/protllama/batch_script/'
sp = spm.SentencePieceProcessor()
sp.Load(tokenizer_path+"protein_8k.model")

True

In [30]:
import torch
def tokenize_batch(batch):
    texts = [sample['sequences'] for sample in batch]
    tokenized_texts = [sp.EncodeAsIds(seq) for seq in texts]
    print(tokenized_texts)
    max_length_in_batch = max(len(t) for t in tokenized_texts)
    padded_tokenized_texts = [t + [0] * (max_length_in_batch - len(t)) for t in tokenized_texts]

    input_ids = torch.tensor(padded_tokenized_texts, dtype=torch.long)

    attention_mask = (input_ids != 0).int()

    data = {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': input_ids.clone()
    }
    return data

#tokenized_dataset = train_dataset.map(tokenize_batch, batched=True)

In [31]:
#print(tokenized_dataset["sequences"][1])

In [32]:
#tokenized_dataset

In [33]:
from torch.utils.data import DataLoader
dataloader = DataLoader(train_dataset,
                     batch_size=2,
                     shuffle=False,
                     drop_last=False,
                     collate_fn=tokenize_batch)

In [34]:
for step, batch in enumerate(dataloader):
    for k, v in batch.items():
        print(k, v)

[[820, 814, 3992, 37, 1313, 7791, 245, 7689, 701, 4163, 6182, 1302, 4844, 7660, 1104, 3455, 2232, 4290, 2886, 2351, 2844, 3768, 4240, 2298, 3576, 1302, 828, 1990, 3031, 6867, 551, 3104, 761, 541, 2541, 2932, 1187, 2101, 4076, 519, 2645, 1104, 828, 2146, 4832, 1891, 868, 1493, 2288, 2410, 1947, 6317, 365, 3000, 2796, 4081, 2571, 3786, 4690, 3698, 965, 834, 1538, 961, 500, 378, 1943, 7364, 3026, 7, 1575, 1191, 3022, 2185, 1396, 5556, 1090, 1841, 3527, 6961, 921, 6227, 38, 406, 49, 4240, 6171, 1932, 412, 506, 842, 958, 1885, 1541, 406, 3602, 2407, 4506, 2106, 129, 1067, 1154, 3093, 496, 6672, 1572, 5982, 2304, 724, 1996, 1065, 1471, 488, 2453, 1161, 6101, 6086, 1326, 262, 1258, 639, 959, 1493, 58, 2120, 1895, 1491, 2227, 795, 721, 1128, 496, 1235, 2004, 576, 3239, 1257, 4403, 2632, 1692, 7227, 586, 1004, 2121, 4750, 574, 3031, 2288, 1492, 1551, 828, 4946, 1697, 4427, 762, 538, 33, 2177, 6725, 5729, 4726, 1268, 492, 1681, 5730, 139, 3346, 3981, 1644, 2463, 7110, 572, 2717, 748, 828, 496, 3

In [55]:
import pickle
with open('/data/rozen/home/e0833634/lama/protllama/original_lama.pkl', 'rb') as f:
    loaded_data_lama = pickle.load(f)
#name = 'togethercomputer/RedPajama-Data-1T-Sample'
#test_sample = loaded_data_lama['train'][:2]
#test_sample_dict = DatasetDict({"train": test_sample})['train']

{'text': ["\\section{Introduction}\n\\label{sec:intro}\n\n\\emph{Gender diversity}, or more often its lack thereof, among participants to\nsoftware development activities has been thoroughly studied in recent years. In\nparticular, the presence of, effects of, and countermeasures for \\emph{gender\n  bias} in Free/Open Source Software (FOSS) have received a lot of attention\nover the past decade~\\cite{david2008fossdevs, qiu2010kdewomen,\n  nafus2012patches, kuechler2012genderfoss, vasilescu2014gender,\n  oneil2016debiansurvey, robles2016womeninfoss, terrell2017gender,\n  zacchiroli2021gender}.  \\emph{Geographic diversity} is on the other hand the\nkind of diversity that stems from participants in some global activity coming\nfrom different world regions and cultures.\n\nGeographic diversity in FOSS has received relatively little attention in scholarly\nworks. In particular, while seminal survey-based and\npoint-in-time medium-scale studies of the geographic origins of FOSS\ncontribut

test_sample = loaded_data_lama['train'][:2]
test_sample

with open("/data/rozen/home/e0833634/lama/protllama/notebooks/text_sample.pkl", "wb") as f:
    pickle.dump(test_sample, f)

In [ ]:
"""below is the original Llama tokenization handling, load your text sample with pickle first"""

In [43]:
import pickle

In [39]:
from functools import partial
from colossalai.nn.lr_scheduler import CosineAnnealingWarmupLR
from colossalai.nn.optimizer import HybridAdam
from torch.optim import Optimizer
from typing import Optional, Tuple
from transformers.models.llama.tokenization_llama import LlamaTokenizer

2023-09-29 12:40:54.760319: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [58]:
tokenizer = LlamaTokenizer.from_pretrained('hf-internal-testing/llama-tokenizer')
# follows fast chat: https://github.com/lm-sys/FastChat/blob/main/fastchat/train/train.py#L257
tokenizer.pad_token = tokenizer.unk_token


normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


train_set = loaded_data_lama['train']
train_set

In [68]:
def tokenize_batch(batch):
    texts = [sample['text'] for sample in batch]
    data = tokenizer(texts, return_tensors="pt", padding='max_length', truncation=True, max_length=1024)
    data['labels'] = data['input_ids'].clone()
    return data

dataloader = DataLoader(train_set, batch_size=2, shuffle=False,
                        drop_last=True,
                        collate_fn=partial(tokenize_batch))

with open("/data/rozen/home/e0833634/lama/protllama/notebooks/llama_dataloader.pkl", "wb") as f:
    pickle.dump(dataloader, f)

In [74]:
with open("/data/rozen/home/e0833634/lama/protllama/notebooks/llama_dataloader.pkl", "rb") as f:
    dataloader = pickle.load(f)

In [75]:
import numpy as np
for step, batch in enumerate(dataloader):
    if step == 1:
        break
    for k, v in batch.items():
        print(k, v)
        print(np.shape(v))

input_ids tensor([[    1,   320,  2042,  ...,   424, 23460, 29889],
        [    1,   320,  2042,  ...,   393,   445, 14581]])
torch.Size([2, 1024])
attention_mask tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
torch.Size([2, 1024])
labels tensor([[    1,   320,  2042,  ...,   424, 23460, 29889],
        [    1,   320,  2042,  ...,   393,   445, 14581]])
torch.Size([2, 1024])
